In [ ]:
from quantum_dataset import QM7, QM7b, QM9, Champs, SuperSet
from quantum_learning import Learn, Selector, ChampSelector
from quantum_model import FFNet
from torch.optim import Adam
from torch.nn import MSELoss, L1Loss

In [ ]:
opt_params = {'lr': 0.001}
model_params = {'D_in': 29*29+29+15+128+32+64+32+64, 'H': 2048, 'D_out': 1, 'model_name': 'funnel'}
qm9_params = {'n': 133885, 
              'features': ['coulomb','mulliken','A','B','C','mu','alpha','homo',
                           'lumo','gap','r2','zpve','U0','U','H','G','Cv'],  
              'dim': 29, 
              'use_pickle': True}
champs_params = {'n': 4658146, 
                 'features': False,
                 'use_h5': False,  
                 'infer': False}
crit_params = {'reduction': 'sum'}
ds_params = {'PrimaryDS': Champs, 'SecondaryDS': QM9, 'p_params': champs_params, 's_params': qm9_params}
sample_params = {'split': .1, 'subset': .1}

Learn(Dataset=SuperSet, Model=FFNet, Sampler=ChampSelector, Optimizer=Adam, Criterion=L1Loss,
      batch_size=32, epochs=20, model_params=model_params, ds_params=ds_params, 
      sample_params=sample_params, opt_params=opt_params, crit_params=crit_params, 
      save_model=False, load_model=False)

In [ ]:
opt_params = {'lr': 0.001}
model_params = {'D_in': 128+32+64+32+64, 'H': 512, 'D_out': 1, 'model_name': 'funnel'}

ds_params = {'n': 4658146, 
             'features': False,
             'use_h5': False,  
             'infer': False}

crit_params = {'reduction': 'sum'}

sample_params = {'split': .1, 'subset': .1}

Learn(Dataset=Champs, Model=FFNet, Sampler=ChampSelector, Optimizer=Adam, Criterion=L1Loss,
      batch_size=2048, epochs=2, model_params=model_params, ds_params=ds_params, 
      sample_params=sample_params, opt_params=opt_params, crit_params=crit_params, 
      save_model=False, load_model=False)